#### Importações necessárias
***

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
import dotenv
dotenv.load_dotenv('../config/.env')


True

#### Extraindo e mostrando os dados dos datasets
***

In [2]:
PATH = '../datasets/dados_clinicos/dados_clinicos.csv'
df_dados_clinicos = pd.read_csv(PATH, sep=';', encoding='latin1')

In [3]:
PATH = '../datasets/dados_estado_regiao/estado_regiao.csv'
df_estado_regiao = pd.read_csv(PATH, sep=';', encoding='latin1')

In [4]:
PATH = '../datasets/dados_pacientes/dados_pacientes.csv'
df_dados_pacientes = pd.read_csv(PATH, sep=';', encoding='latin1')

In [5]:
df_dados_pacientes.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430


In [6]:
df_dados_clinicos.head()

,id_cliente,peso,colesterol,genero
0,1,152.7,203.9,Masculino
1,2,184.8,233.8,Masculino
2,3,152.1,210.4,Masculino
3,4,139.8,196.8,Masculino
4,5,182.4,215.8,Feminino


In [7]:
df_estado_regiao.head()

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil


#### Verificando e corrigindo os dados duplicados
***

In [8]:
def verifica_dados_duplicados(nome_dataframe, dataframe):
    qtd = dataframe.duplicated().sum()

    print(f'O dataframe {nome_dataframe} possui {qtd} dados duplicados')

In [9]:
verifica_dados_duplicados('dados_clinicos', df_dados_clinicos)
verifica_dados_duplicados('dados_pacientes', df_dados_pacientes)
verifica_dados_duplicados('dados_estado_regiao', df_estado_regiao)

O dataframe dados_clinicos possui 301 dados duplicados
O dataframe dados_pacientes possui 85 dados duplicados
O dataframe dados_estado_regiao possui 9 dados duplicados


In [10]:
df_dados_pacientes = df_dados_pacientes.drop_duplicates()
verifica_dados_duplicados('df_dados_pacientes', df_dados_pacientes)

O dataframe df_dados_pacientes possui 0 dados duplicados


In [11]:
df_dados_clinicos = df_dados_clinicos.drop_duplicates()
verifica_dados_duplicados('df_dados_clinicos', df_dados_clinicos)

O dataframe df_dados_clinicos possui 0 dados duplicados


In [12]:
df_estado_regiao = df_estado_regiao.drop_duplicates()
verifica_dados_duplicados('df_estado_regiao', df_estado_regiao)

O dataframe df_estado_regiao possui 0 dados duplicados


In [13]:
lista_dataframes = [
    ('dados_clinicos', df_dados_clinicos),
    ('dados_pacientes', df_dados_pacientes),
    ('dados_estado_regiao', df_estado_regiao)
]

for nome_dataframe, dataframe in lista_dataframes:
           
    print('--' * 50)
    verifica_dados_duplicados(nome_dataframe, dataframe)
print('--' * 50)

----------------------------------------------------------------------------------------------------
O dataframe dados_clinicos possui 0 dados duplicados
----------------------------------------------------------------------------------------------------
O dataframe dados_pacientes possui 0 dados duplicados
----------------------------------------------------------------------------------------------------
O dataframe dados_estado_regiao possui 0 dados duplicados
----------------------------------------------------------------------------------------------------


#### Verificando e corrigindo os dados ausentes dos dados_pacientes
***

In [14]:
def verifica_dados_ausentes(nome_dataframe, dataframe):
    dados_ausentes = dataframe.isna().sum()
    colunas_com_ausentes = dados_ausentes[dados_ausentes > 0]
    
    print('--' * 50)
    
    if not colunas_com_ausentes.empty:
        print(f'O dataframe {nome_dataframe} possui dados ausentes.\nLista de colunas:')
        return print(colunas_com_ausentes)
    else:
        print(f'O dataframe {nome_dataframe} não possui dados ausentes.')

In [15]:
verifica_dados_ausentes('df_dados_pacientes', df_dados_pacientes)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_pacientes possui dados ausentes.
Lista de colunas:
classe_trabalho    387
qtde_filhos         10
dtype: int64


In [16]:
filtro = df_dados_pacientes['classe_trabalho'].isna()
index_ausentes_classe_trabalho = df_dados_pacientes[filtro].index
df_dados_pacientes.loc[index_ausentes_classe_trabalho]

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
61,62,32,NaN,Ensino Fundamental Incompleto,12,União Estável,Branco,4.0,1100
69,70,25,NaN,Ensino Médio Completo,1,Solteiro,Branco,3.0,3450
127,128,35,NaN,Ensino Médio Incompleto,1,Casado,Amarelo,4.0,1100
148,149,43,NaN,Ensino Médio Completo,21,Divorciado,Branco,0.0,2490
153,154,52,NaN,Ensino Médio Incompleto,18,Divorciado,Branco,1.0,1100
...,...,...,...,...,...,...,...,...,...
8016,7932,51,NaN,Ensino Médio Completo,4,Divorciado,Branco,2.0,2660
8037,7953,19,NaN,Ensino Médio Completo,1,Solteiro,Branco,0.0,2581
8045,7961,30,NaN,Ensino Fundamental Completo,10,Solteiro,Branco,4.0,1100
8073,7989,20,NaN,Ensino Médio Completo,1,Solteiro,Branco,2.0,3610


In [17]:
moda_classe_trabalho = df_dados_pacientes['classe_trabalho'].mode()[0]
print(moda_classe_trabalho)
df_dados_pacientes['classe_trabalho'] = df_dados_pacientes['classe_trabalho'].fillna(value=moda_classe_trabalho)

Funcionário Setor Privado


In [18]:
df_dados_pacientes.loc[index_ausentes_classe_trabalho]

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
61,62,32,Funcionário Setor Privado,Ensino Fundamental Incompleto,12,União Estável,Branco,4.0,1100
69,70,25,Funcionário Setor Privado,Ensino Médio Completo,1,Solteiro,Branco,3.0,3450
127,128,35,Funcionário Setor Privado,Ensino Médio Incompleto,1,Casado,Amarelo,4.0,1100
148,149,43,Funcionário Setor Privado,Ensino Médio Completo,21,Divorciado,Branco,0.0,2490
153,154,52,Funcionário Setor Privado,Ensino Médio Incompleto,18,Divorciado,Branco,1.0,1100
...,...,...,...,...,...,...,...,...,...
8016,7932,51,Funcionário Setor Privado,Ensino Médio Completo,4,Divorciado,Branco,2.0,2660
8037,7953,19,Funcionário Setor Privado,Ensino Médio Completo,1,Solteiro,Branco,0.0,2581
8045,7961,30,Funcionário Setor Privado,Ensino Fundamental Completo,10,Solteiro,Branco,4.0,1100
8073,7989,20,Funcionário Setor Privado,Ensino Médio Completo,1,Solteiro,Branco,2.0,3610


In [19]:
filtro = df_dados_pacientes['qtde_filhos'].isna()
index_ausentes_qtde_filhos = df_dados_pacientes[filtro].index
df_dados_pacientes.loc[index_ausentes_qtde_filhos]

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
12,13,23,Funcionário Setor Privado,Pós-Graduado,22,Solteiro,Branco,NaN,5316
20,21,40,Funcionário Setor Privado,Doutorado,8,Casado,Branco,NaN,12535
189,190,38,Funcionário Setor Privado,Ensino Médio Completo,27,Divorciado,Negro,NaN,2369
233,234,28,Funcionário Setor Privado,Mestrado,26,Solteiro,Branco,NaN,10671
295,296,37,MEI,Ensino Médio Completo,18,Divorciado,Branco,NaN,4300
358,359,26,Autônomo,Ensino Fundamental Completo,24,Casado,Branco,NaN,1100
359,360,36,Funcionário Setor Privado,Ensino Fundamental Completo,11,Casado,Branco,NaN,1100
360,361,62,Funcionário Setor Privado,Mestrado,7,Casado,Branco,NaN,6072
362,363,43,Funcionário Setor Privado,Ensino Médio Completo,12,Divorciado,Branco,NaN,2240
402,403,28,Funcionário Setor Privado,Ensino Fundamental Incompleto,27,Casado,Amarelo,NaN,1100


In [20]:
moda_qtde_filhos = df_dados_pacientes['qtde_filhos'].median()
print(moda_qtde_filhos)
df_dados_pacientes['qtde_filhos'] = df_dados_pacientes['qtde_filhos'].fillna(value=moda_qtde_filhos)

2.0


In [21]:
df_dados_pacientes.loc[index_ausentes_qtde_filhos]

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
12,13,23,Funcionário Setor Privado,Pós-Graduado,22,Solteiro,Branco,2.0,5316
20,21,40,Funcionário Setor Privado,Doutorado,8,Casado,Branco,2.0,12535
189,190,38,Funcionário Setor Privado,Ensino Médio Completo,27,Divorciado,Negro,2.0,2369
233,234,28,Funcionário Setor Privado,Mestrado,26,Solteiro,Branco,2.0,10671
295,296,37,MEI,Ensino Médio Completo,18,Divorciado,Branco,2.0,4300
358,359,26,Autônomo,Ensino Fundamental Completo,24,Casado,Branco,2.0,1100
359,360,36,Funcionário Setor Privado,Ensino Fundamental Completo,11,Casado,Branco,2.0,1100
360,361,62,Funcionário Setor Privado,Mestrado,7,Casado,Branco,2.0,6072
362,363,43,Funcionário Setor Privado,Ensino Médio Completo,12,Divorciado,Branco,2.0,2240
402,403,28,Funcionário Setor Privado,Ensino Fundamental Incompleto,27,Casado,Amarelo,2.0,1100


#### Verificando e corrigindo os dados ausentes dos dados_clínicos
***

In [22]:
verifica_dados_ausentes('df_dados_clinicos', df_dados_clinicos)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_clinicos possui dados ausentes.
Lista de colunas:
peso      13
genero    10
dtype: int64


In [23]:
filtro = df_dados_clinicos['peso'].isna()
index_ausentes_peso = df_dados_clinicos[filtro].index
df_dados_clinicos.loc[index_ausentes_peso]

,id_cliente,peso,colesterol,genero
10224,9924,NaN,201.1,Feminino
10225,9925,NaN,225.2,Masculino
10230,9930,NaN,134.9,Feminino
10231,9931,NaN,237.5,Feminino
10243,9943,NaN,219.3,Feminino
10244,9944,NaN,246.2,NaN
10248,9948,NaN,184.0,Masculino
10251,9951,NaN,182.5,Feminino
10262,9962,NaN,115.9,Masculino
10270,9970,NaN,173.1,Masculino


In [24]:
mediana_peso = df_dados_clinicos['peso'].median()
print(mediana_peso)
df_dados_clinicos['peso'] = df_dados_clinicos['peso'].fillna(value=mediana_peso)

124.6


In [25]:
filtro = df_dados_clinicos['genero'].isna()
index_ausentes_genero = df_dados_clinicos[filtro].index
df_dados_clinicos.loc[index_ausentes_genero]

,id_cliente,peso,colesterol,genero
10227,9927,177.6,220.3,NaN
10228,9928,149.3,199.3,NaN
10233,9933,65.3,132.8,NaN
10234,9934,180.6,214.1,NaN
10237,9937,103.6,160.8,NaN
10240,9940,81.6,165.8,NaN
10241,9941,154.9,213.6,NaN
10242,9942,96.3,163.0,NaN
10244,9944,124.6,246.2,NaN
10245,9945,51.4,130.5,NaN


In [26]:
moda_genero = df_dados_clinicos['genero'].mode()[0]
print(moda_genero)
df_dados_clinicos['genero'] = df_dados_clinicos['genero'].fillna(value=moda_genero)

Feminino


In [27]:
df_dados_clinicos.loc[index_ausentes_genero]

,id_cliente,peso,colesterol,genero
10227,9927,177.6,220.3,Feminino
10228,9928,149.3,199.3,Feminino
10233,9933,65.3,132.8,Feminino
10234,9934,180.6,214.1,Feminino
10237,9937,103.6,160.8,Feminino
10240,9940,81.6,165.8,Feminino
10241,9941,154.9,213.6,Feminino
10242,9942,96.3,163.0,Feminino
10244,9944,124.6,246.2,Feminino
10245,9945,51.4,130.5,Feminino


In [28]:
verifica_dados_ausentes('df_dados_clinicos', df_dados_clinicos)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_clinicos não possui dados ausentes.


#### Verificando e corrigindo os dados ausentes dos dados_estado_regiao
***

In [29]:
verifica_dados_ausentes('df_estado_regiao', df_estado_regiao)

----------------------------------------------------------------------------------------------------
O dataframe df_estado_regiao não possui dados ausentes.


In [30]:
df_estado_regiao.shape

(27, 5)

In [31]:
df_estado_regiao.head(27)

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil
5,6,CE,Ceará,Nordeste,Brasil
6,7,DF,Distrito Federal,Centro-Oeste,Brasil
7,8,ES,Espírito Santo,Sudeste,Brasil
8,9,GO,Goiás,Centro-Oeste,Brasil
9,10,MA,Maranhão,Nordeste,Brasil


In [32]:
print(df_estado_regiao.dtypes)
print('--' * 50)
print(df_dados_clinicos.dtypes)
print('--' * 50)
print(df_dados_pacientes.dtypes)

id_estado     int64
sigla        object
estado       object
regiao       object
pais         object
dtype: object
----------------------------------------------------------------------------------------------------
id_cliente      int64
peso          float64
colesterol    float64
genero         object
dtype: object
----------------------------------------------------------------------------------------------------
id_cliente           int64
idade                int64
classe_trabalho     object
escolaridade        object
id_estado            int64
estado_civil        object
raca                object
qtde_filhos        float64
salario              int64
dtype: object


### Conexão com o Banco de Dados
***

In [33]:
def conectar_banco_mysql():
    try:
        user = os.environ['user_db']
        password = os.environ['password_db']
        host = os.environ['host']

        conexao = f'mysql://{user}:{password}@{host}'
        engine = create_engine(conexao)
        conn = engine.connect()
        print('Conexão Realizada com Sucesso!')
        return conn
    except Exception as e:
        print(f'Não foi possível realizar a conexão. Erro: {e}')

In [34]:
conn = conectar_banco_mysql()

Conexão Realizada com Sucesso!


#### Criação do Banco de Dados
***

In [35]:
try:
    database = 'enunciado_clinica'
    query = f'create schema if not exists {database}'
    conn.execute(text(query))
    print('Banco de dados criado com sucesso!')
    conn.commit()
except Exception as e:
    print(f'Não foi possível criar o banco de dados. Erro: {e}')

Banco de dados criado com sucesso!


#### Executa o Script de Criação das Tabelas e Seus Relacionamentos

In [36]:
with open('../projetos/script_create_DB.sql') as file:
    query = file.read()

try:
    conn.execute(text(query))
    conn.commit()
    print('Tabelas criadas com sucesso!')
except Exception as e:
    print(f'Não foi possível criar as tabelas do banco de dados. Erro: {e}')

Tabelas criadas com sucesso!


#### Mesclando os dataframes resultantes dos datasets
***

In [37]:
df_primeira_mesclagem = pd.merge(left=df_dados_pacientes, right=df_dados_clinicos, left_on='id_cliente', right_on='id_cliente', how='inner')

In [38]:
df_completo = pd.merge(left=df_primeira_mesclagem, right=df_estado_regiao, left_on='id_estado', right_on='id_estado', how='inner')

#### Corrigindo os floats para 2 casas decimais
***

In [44]:
df_dados_pacientes['salario'] = df_dados_pacientes['salario'].round(2)
df_dados_clinicos['peso'] = df_dados_clinicos['peso'].round(2)
df_dados_clinicos['colesterol'] = df_dados_clinicos['colesterol'].round(2)

#### Mostrando o resultado da mesclagem
***

In [40]:
df_completo.columns

Index(['id_cliente', 'idade', 'classe_trabalho', 'escolaridade', 'id_estado',
       'estado_civil', 'raca', 'qtde_filhos', 'salario', 'peso', 'colesterol',
       'genero', 'sigla', 'estado', 'regiao', 'pais'],
      dtype='object')

In [42]:
df_completo.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario,peso,colesterol,genero,sigla,estado,regiao,pais
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754,152.7,203.9,Masculino,AL,Alagoas,Nordeste,Brasil
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923,184.8,233.8,Masculino,SC,Santa Catarina,Sul,Brasil
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100,152.1,210.4,Masculino,AM,Amazonas,Norte,Brasil
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100,139.8,196.8,Masculino,SC,Santa Catarina,Sul,Brasil
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430,182.4,215.8,Feminino,PB,Paraíba,Nordeste,Brasil


In [43]:
print(df_estado_regiao.dtypes)
print('--' * 50)
print(df_dados_clinicos.dtypes)
print('--' * 50)
print(df_dados_pacientes.dtypes)

id_estado     int64
sigla        object
estado       object
regiao       object
pais         object
dtype: object
----------------------------------------------------------------------------------------------------
id_cliente      int64
peso          float64
colesterol    float64
genero         object
dtype: object
----------------------------------------------------------------------------------------------------
id_cliente           int64
idade                int64
classe_trabalho     object
escolaridade        object
id_estado            int64
estado_civil        object
raca                object
qtde_filhos        float64
salario              int64
dtype: object
